In [1]:
import pandas as pd
import numpy as np
import random as rand
import tqdm
import networkx as nx
import pickle as pkl
import gower
import matplotlib.pyplot as plt
import methods
from methods import topK, findGroups, prototype, ranking, getAttributes, Q, Q2, Discovery

In [2]:
from torch_geometric.datasets import Twitch

# Load the Tolokers dataset
dataset = Twitch(root='./data/Twitch', name='EN')

# Access the first graph in the dataset
data = dataset[0]

In [19]:
dataset = Twitch(root='./data/Twitch', name='EN')

# Custom wrapper to simulate GraphPropPredDataset structure
class CustomGraphDataset:
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        # Extract a graph from the dataset
        graph_data = self.dataset[idx]
        
        # Convert to the desired dictionary format similar to GraphPropPredDataset
        graph = {
            'edge_index': graph_data.edge_index,  # Edge index
            'num_nodes': graph_data.num_nodes,  # Number of nodes
            'node_feat': graph_data.x  # Node features (if available)
        }
        
        label = graph_data.y
        return graph, label

    def __len__(self):
        return len(self.dataset)

# Instantiate the custom dataset
custom_dataset = CustomGraphDataset(dataset)
graph, label = custom_dataset[0]

# Access edge index, number of nodes, and node features
edge_index = graph['edge_index']
num_nodes = graph['num_nodes']

In [18]:
edge_index = data.edge_index 
num_nodes = data.num_nodes 
G = nx.Graph()

G.add_nodes_from(range(num_nodes))

edges = list(zip(edge_index[0].tolist(), edge_index[1].tolist()))  # Convert tensor to list of tuples
G.add_edges_from(edges)

In [28]:
label

tensor([1, 0, 1,  ..., 0, 0, 1])

In [31]:
attributes = graph['node_feat']
lu = pd.DataFrame(attributes)
lu['target'] = label == 1 # use the label binary explicit language as target variable?
lu.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-0.236669,-0.230716,-0.160547,-0.198200,-0.196495,-0.179999,-0.538649,-0.411488,-0.162248,-0.221553,...,4.198101,-0.214107,-0.355245,-0.175422,-0.212333,-0.375776,1.320289,-0.255846,-0.183876,True
1,-0.236669,-0.214971,-0.160547,-0.217247,-0.212379,-0.203848,-0.538649,-0.411488,-0.159587,-0.223381,...,-0.239307,-0.214107,3.376396,-0.175422,-0.213861,2.662601,1.357220,-0.253838,-0.168536,False
2,-0.236669,-0.204347,-0.099350,-0.217247,-0.212379,-0.203848,1.573361,-0.404173,-0.162248,-0.072138,...,-0.239307,-0.211431,-0.355245,-0.175422,-0.213861,-0.375776,-0.634779,-0.255846,-0.183876,True
3,-0.182715,-0.220977,-0.160547,-0.217247,-0.212379,-0.203848,-0.538649,-0.411488,-0.155285,-0.223381,...,-0.239307,-0.214107,-0.355245,-0.175422,-0.213861,-0.375776,1.354141,-0.255055,-0.183876,False
4,-0.236669,-0.230716,-0.160547,-0.217247,-0.212379,-0.199876,-0.538649,-0.411488,-0.162248,-0.223381,...,-0.239307,-0.214107,-0.355245,-0.175422,-0.213861,2.665692,1.231107,-0.255846,-0.183876,True


In [33]:
result = findGroups(G, 20, lu)
result # this will take loooong on my PC